# Woodall Numbers
A [Woodall number](https://en.wikipedia.org/wiki/Woodall_number) is an integer, $W$, that satisfies $k2^{k}-1=W\in\mathbb{Z}^+$ for some $k\in\mathbb{Z}^+$. The OEIS number for the sequence of Woodall numbers is [A003261](https://oeis.org/A003261).
## Determining whether $z\in\mathbb{Z}^+$ Is a Woodall
Given a candidate, $z\in\mathbb{Z}^+$, it is simple to determine whether $z$ is a Woodall, requiring only incrementing the value of $k$ in a trial-and-error approach. This can become computationally infeasible as $z$ grows, however.

This notebook will show that there is an upper bound in the search range of $k$ and it will show how to implement it in Python– with _no_ modules imported, at that! 

## The Code
The following function will return a Boolean corresponding to whether the input, `z`, is a Woodall number.
```python
def is_woodall(z):
    """ Is there an int, k, such that
    `z` = k * 2**k - 1?
    """
    if z < 1:
        return False
    w = z + 1
    woodall = lambda k, w=w: k * 2 ** k == w
    range_of_k_to_check = range(1, 4) if z < 64 else \
        range(1, int(pow(w, 1/3)) + 1)  # 1,..., floor of cube root of z + 1
    return any(map(woodall, range_of_k_to_check))
```
### Demonstration
Take the first, say, 100 integers, as arguments to the function. Only 1, 7, 23, and 63 should return `True`:
```python
>>> woodalls = {1, 7, 23, 63}
>>> not any(is_woodall(z) for z in set(range(100)) - woodalls)
True

>>> all(is_woodall(z) for z in woodalls)
True
```
_**Note**_: As with any Python code, unit tests should be included. These can be found below.
### Explanation
There are a few talking points around `is_woodall`, especially the `range_of_k_to_check` for input greater than 63.
1. Woodall numbers start with 1, thus any `z` smaller is not Woodall
2. It is computationally more efficient to check against a variable that has the 1 added than to check if `(k*2**k)-1` is Woodall for every `k`
3. The `woodall` function uses Python's inline function definition syntax, `lambda`, to create a function that returns a Boolean for every `k` and `w` that are passed to it. By specifying `lambda k, w=w:`, `w` in the scope of `is_woodall` is "pinned" in the scope of `woodall`, just as if it were a normal function definition; i.e. `def woodall(k, w=w):`
4. Why is 64 a special value of `w`? The table below, and the proof in the subsequent section, will justify the splitting up of `k`s to check
5. Python's `any` function utilizes [short-circuit evaluation](https://en.wikipedia.org/wiki/Short-circuit_evaluation), meaning that the first element of the argument that is truthy will cause the `return`ing of `True` before all of `range_of_k_to_check` is evaluated. Furthermore, the `map` object produced by this call is lazily-evaluated, conserving memory due to a potentially large range of `int`s
6. Lastly, if `range_of_k_to_check` takes the `else` branch in the ternary expression, `1` is added to the second argument of `range` so that the value `int(pow(w, 1/3))` itself is an element of the `range` object

## The Math
When first approaching the problem of determining the Woodallity of a given integer, $W$, the natural approach is to start with a small value of $k$, calculate $k2^{k}-1$ and increase $k$ if this calculation does not yield $W$. This is straightforward, and it is an exhaustive algorithm because as soon as the value $k2^{k}-1$ exceeds $W$, it is established that $W$ is not Woodall (due to $f(k)=k$ and $g(k) = 2^k$ being monotone increasing functions). However, there is a better way.

### The Proposition
The reduction of the search space of $k$ claimed by the `is_woodall` function is the following:
$$\text{Given} \ 63 \leq W \in \mathbb{Z},\ \exists k\in\mathbb{Z}^+ \text{such that} \ W=k2^{k}-1\implies k \leq \sqrt[3]{W+1}$$
i.e. that an algorithm checking for Woodallity need only try values of $k$ up to the cube root of one more than the candidate, $W$.

### The Intuition
Tabulating the first few values of $k$ and the $W$ that they would produce, it turns out that for every $k$ in the table _except_ $k=3$, $k^3 \leq w$ (where $w$ is equal to $W+1$)

| $k$ | $k^3$ | $w$ | $W$ |
|---|------|------|------|
| 1 | 1  | 2  | 1  |
| 2 | 8  | 8  | 7  |
| 3 | 27 | 24 | 23 |
| 4 | 64 | 64 | 63 |
| 5 | 125 | 160 | 159 |
| 6 | 136 | 384 | 383 |

This is why `is_woodall` changes search behavior at 64: for every $k \geq 4$, $k^3 > w$, with equality for $k=4$. For a given $W$, therefore, one only need search up to $\lfloor\sqrt[3]{W+1}\rfloor$ 

### The Proof
The proof of the above proposition relies on the following lemma.
#### The Lemma
$$\forall k \in \mathbb{N}\backslash\{0,1,2,3\}, \ \ k^2 \leq 2^k$$

In [36]:
%%latex
\begin{align}
Proof:
    \text{Assume that } \ 2^k \geq k^2 \ \text{for} \ k \geq 4. \ \text{The base case is:} \ 2^4 = 16 \geq 16 = 4^2. \text{Then,} \\
    \ 2^k &\geq k^2 \\
    \implies 2(2^{k}) &\geq 2k^2 \\
    \implies 2^{k+1} &\geq 2k^2 \geq k^2 + 2k + 1 \ \ \ \ \ \ \ \ \ \ \star \\
    \implies 2^{k+1} &\geq k^2 + 2k + 1 = (k+1)^2 \\
    
\text{completing the induction step. Justification of} \ \star \text{is available in the appendix}
\end{align}

<IPython.core.display.Latex object>

#### Proof of the Proposition
This proof does not proceed via induction, but by [direct proof](https://en.wikipedia.org/wiki/Direct_proof)

In [64]:
%%latex
\begin{align}
Proof:
    \text{Put} \ W\geq 63, \ \text{and assume that} \ \exists k\in\mathbb{Z}^+ \ \text{such that} \ W=k2^{k}-1. \text{Then,} \\
    k^2 &\leq 2^k \\
    \implies k^3 &\leq k2^k \\
    \implies k^3 - 1 &\leq k2^{k} - 1 \\
    \implies k^3 - 1 &\leq W \\
    \implies k &\leq \sqrt[3]{W+1} \ \ \ \ \ \ \square
\end{align}

<IPython.core.display.Latex object>

## Testing
Testing properties of an infinite set with finite hardware is challenging, but with the combination of a source of authority (On-line Encyclopedia for Integer Sequences) and the naive algorithm (using this just passes the buck of testing), it is possible to be reasonably sure that `is_woodall` is implemented correctly. This is because the [`hypothesis`](https://hypothesis.readthedocs.io/en/latest/) Python package implements [property-based tests](https://hypothesis.works/articles/what-is-property-based-testing/). By using the `@given` decorators in the tests below, `hypothesis` will run a test suite of inputs through `is_woodall` that aims to capture the various edge cases of the given input type. From the documentation,
>Hypothesis doesn’t just find any counter-example to your tests, it knows how to simplify the examples it finds to produce small easy to understand ones.

Thus, there are two tests below. The first, `test_woodall_1`, does a set lookup on the published Woodall numbers on the [OEIS entry](https://oeis.org/A003261) for the Woodall numbers. If this test passes, it is an indication that the logic of `is_woodall` holds for small input values. The second test relies on `naive_is_woodall` to assert the Woodallity of the `int`s generated by `hypothesis.strategies.integers`. This test does not have a `max_value` set in the strategy passed to `hypothesis`, so that it can test the full range of `int`s that Python handles. This test takes a considerable amount of time: However, that both tests pass is a strong indication that `is_woodall` passes muster. 

In [ ]:
from hypothesis import given, strategies as st

def naive_is_woodall(z):
    """An implementation of the "naive" 
    algorithm to determine Woodallity of z"""
    k = 1
    w = z + 1
    candidate = k*2**k
    while candidate <= w:
        if w == candidate:
            return True
        k += 1
        candidate = k*2**k
        continue
    else:
        return False

@given(st.integers(max_value=7.6e9))
def test_is_woodall_1(z):
    woodalls = \
    {1, 7, 23, 63, 159, 383, 895, 
     2047, 4607, 10239, 22527, 
     49151, 106495, 229375, 491519,
     1048575, 2228223, 4718591, 
     9961471, 20971519, 44040191, 
     92274687, 192937983, 402653183, 
     838860799, 1744830463, 3623878655, 7516192767}
    if z not in woodalls:
        assert not is_woodall(z)
    else:
        assert is_woodall(z)
      
@given(st.integers())
def test_is_woodall_2(z):
    if z < 1:
        assert not is_woodall(z)
    if naive_is_woodall(z):
        assert is_woodall(z)
    else:
        assert not is_woodall(z)

# Appendix
The statement denoted by ($\star$) in the proof of the lemma above is more formally stated as:
$$\forall n\in\mathbb{N}\backslash\{0,1,2\}, \ \ 2n^2 \geq n^2 + 2n + 1$$

In [63]:
%%latex
\begin{align}
Proof: \text{Put} \ n \geq 3. \ \text{Then,} \\
    n &\geq \sqrt{2} + 1 \\
    \implies n-1 &\geq \sqrt{2} \\
    \implies (n-1)^2 &\geq 2 \\
    \implies (n-1)^2 -1 &\geq 1 \\
    \implies n^2-2n-1 &\geq 0 \\
    \implies n^2 &\geq 2n + 1 \\
    \implies 2n^2 &\geq n^2 + 2n + 1 \ \ \ \square
\end{align}

<IPython.core.display.Latex object>